In [18]:
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key="pcsk_5prr2Q_2DTFBLH13m7TQ2x4phRThATQew6yUB4o95L6RsWPuzZwV1Ggc3e3rxd6FqHKPe7"
)

# Now do stuff
if 'input-output-pair' not in pc.list_indexes().names():
    pc.create_index(
        name='input-output-pair',
        dimension=384,
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )




In [19]:
!pip install pinecone-client


In [20]:
import json

# Open and read the JSON file
with open("/content/OETReadingPartA.json", "r") as file:
    data = json.load(file)

# Now `data` is a dictionary
print(data)


{'training_data': [{'input': 'Create a reading comprehension task suitable for the OET Reading Test, specifically for Part A. The task should involve a cohesive passage about managing asthma and include 20 questions based on this passage.', 'output': {'passage': '{ "title": "Managing Asthma: A Comprehensive Approach", "sections": [ { "section_title": "Understanding Asthma", "content": "Asthma is a chronic respiratory condition characterized by inflammation and narrowing of the airways. Common symptoms include wheezing, shortness of breath, chest tightness, and coughing. It can be triggered by allergens, pollution, exercise, and respiratory infections. Effective management involves identifying triggers, using medications, and adopting lifestyle changes." }, { "section_title": "Medications for Asthma Control", "content": "Asthma medications are categorized into two main types: relievers and controllers. Relievers, such as short-acting beta-agonists (SABAs), provide quick relief during as

In [21]:
from pinecone import Pinecone

# pc = Pinecone(api_key="********-****-****-****-************")
index = pc.Index("input-output-pair")

In [24]:
from sentence_transformers import SentenceTransformer

# Load your model for embeddings (optional if using metadata alone)
model = SentenceTransformer("all-MiniLM-L6-v2")
for i, item in enumerate(data["training_data"]):
    unique_id = f"input_{i+1}"
    input_text = item["input"]
    output_text = item["output"]

    # Convert output to a JSON string if it's a complex object
    output_text = json.dumps(output_text) if isinstance(output_text, dict) else str(output_text)
    embedding = model.encode(input_text).tolist()

    # Upsert the data to Pinecone
    index.upsert([(unique_id, embedding, {"input": input_text, "output": output_text})])

In [25]:
result = index.fetch(ids=["input_1"])  # Replace with desired ID
print(result)


{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'input_1': {'id': 'input_1',
                         'metadata': {'input': 'Create a reading comprehension '
                                               'task suitable for the OET '
                                               'Reading Test, specifically for '
                                               'Part A. The task should '
                                               'involve a cohesive passage '
                                               'about managing asthma and '
                                               'include 20 questions based on '
                                               'this passage.',
                                      'output': '{"passage": "{ \\"title\\": '
                                                '\\"Managing Asthma: A '
                                                'Comprehensive Approach\\", '
                                                '\\"sections\\": [ { '
 

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the model (use the same model you used for initial embeddings)
model = SentenceTransformer("all-MiniLM-L6-v2")

# New input for similarity search
new_input = "Describe the symptoms and treatment for asthma."

# Generate the embedding for the new input
new_input_embedding = model.encode(new_input).tolist()


In [28]:
# Number of results to retrieve
top_k = 1

# Perform similarity search in Pinecone
result = index.query(vector=new_input_embedding, top_k=top_k, include_metadata=True)

# Print the top results
for match in result["matches"]:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Input: {match['metadata']['input']}")
    print(f"Output: {match['metadata']['output']}")
    print("=" * 50)


ID: input_1
Score: 1.23419952
Input: Create a reading comprehension task suitable for the OET Reading Test, specifically for Part A. The task should involve a cohesive passage about managing asthma and include 20 questions based on this passage.
Output: {"passage": "{ \"title\": \"Managing Asthma: A Comprehensive Approach\", \"sections\": [ { \"section_title\": \"Understanding Asthma\", \"content\": \"Asthma is a chronic respiratory condition characterized by inflammation and narrowing of the airways. Common symptoms include wheezing, shortness of breath, chest tightness, and coughing. It can be triggered by allergens, pollution, exercise, and respiratory infections. Effective management involves identifying triggers, using medications, and adopting lifestyle changes.\" }, { \"section_title\": \"Medications for Asthma Control\", \"content\": \"Asthma medications are categorized into two main types: relievers and controllers. Relievers, such as short-acting beta-agonists (SABAs), provid